In [18]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
import numpy as np
import scipy.sparse as sp
import pandas as pd

In [27]:
ratings = pd.read_csv("../datasets/movielens/sorted_ratings.csv", header=None)

In [45]:
mvGraph = dgl.DGLGraph()

/home/rustambaku13/miniconda3/envs/flink/lib/python3.8/site-packages/dgl/heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'


In [52]:
mvGraph.add_nodes(np.unique(np.concatenate([ratings[0].unique(), ratings[0].unique()])).size)

In [58]:
mvGraph.add_edges(ratings[0].values, ratings[1].values)

In [67]:
mvGraph.ndata['x'] = torch.rand((mvGraph.num_nodes(), 7))

In [77]:
sageConv = dgl.nn.SAGEConv(7, 7, aggregator_type="mean")

In [106]:
for j in sageConv.modules():
    print(j)
    for i in j.parameters():
        print(i, i.grad)

SAGEConv(
  (feat_drop): Dropout(p=0.0, inplace=False)
  (fc_self): Linear(in_features=7, out_features=7, bias=False)
  (fc_neigh): Linear(in_features=7, out_features=7, bias=False)
)
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0.], requires_grad=True) tensor([1., 1., 1., 1., 1., 1., 1.])
Parameter containing:
tensor([[ 0.6413, -0.2408, -0.8501, -0.4668, -0.5369,  0.3948, -0.7702],
        [-0.7609,  0.5340, -0.1975, -0.5465,  0.1076,  0.3526,  0.6505],
        [ 0.7870, -0.5753, -0.7760,  0.0852, -0.7697,  0.6718,  0.7048],
        [ 0.8213,  0.5061, -0.8415,  0.8914,  0.7798,  0.6856, -0.5754],
        [ 0.4530, -0.4652, -0.8454,  0.6100,  0.7065, -0.1291, -0.7999],
        [ 0.7116, -0.0092, -0.5687, -0.7556,  0.3323, -0.3025, -0.1600],
        [ 0.8966, -0.5762,  0.5154, -0.8767,  0.8520,  0.0593,  0.6400]],
       requires_grad=True) tensor([[0.6045, 0.6315, 0.3701, 0.3258, 0.5585, 0.4917, 0.4233],
        [0.6045, 0.6315, 0.3701, 0.3258, 0.5585, 0.4917, 0.4233],
       

In [151]:
embedding = sageConv(mvGraph, mvGraph.ndata['x'])

In [152]:
gradients = torch.zeros((mvGraph.num_nodes(), 7))
gradients[209169,:] = 1

In [159]:
embedding.backward(gradients)

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [154]:
print(sageConv)
for i in sageConv.parameters():
    print(i,i.grad)

SAGEConv(
  (feat_drop): Dropout(p=0.0, inplace=False)
  (fc_self): Linear(in_features=7, out_features=7, bias=False)
  (fc_neigh): Linear(in_features=7, out_features=7, bias=False)
)
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0.], requires_grad=True) tensor([2., 2., 2., 2., 2., 2., 2.])
Parameter containing:
tensor([[ 0.6413, -0.2408, -0.8501, -0.4668, -0.5369,  0.3948, -0.7702],
        [-0.7609,  0.5340, -0.1975, -0.5465,  0.1076,  0.3526,  0.6505],
        [ 0.7870, -0.5753, -0.7760,  0.0852, -0.7697,  0.6718,  0.7048],
        [ 0.8213,  0.5061, -0.8415,  0.8914,  0.7798,  0.6856, -0.5754],
        [ 0.4530, -0.4652, -0.8454,  0.6100,  0.7065, -0.1291, -0.7999],
        [ 0.7116, -0.0092, -0.5687, -0.7556,  0.3323, -0.3025, -0.1600],
        [ 0.8966, -0.5762,  0.5154, -0.8767,  0.8520,  0.0593,  0.6400]],
       requires_grad=True) tensor([[0.6963, 0.2898, 0.6288, 0.0531, 0.9924, 0.6635, 0.5498],
        [0.6963, 0.2898, 0.6288, 0.0531, 0.9924, 0.6635, 0.5498],
       

In [147]:
sageConv.zero_grad()

In [158]:
mvGraph.sample_neighbors(209169, -1).ndata

{'x': tensor([[0.6045, 0.6315, 0.3701,  ..., 0.5585, 0.4917, 0.4233],
        [0.1209, 0.0220, 0.9092,  ..., 0.5982, 0.9966, 0.7096],
        [0.9955, 0.9801, 0.4451,  ..., 0.5891, 0.2126, 0.5948],
        ...,
        [0.6963, 0.2898, 0.6288,  ..., 0.9924, 0.6635, 0.5498],
        [0.7448, 0.0271, 0.0137,  ..., 0.0751, 0.3804, 0.9385],
        [0.8445, 0.3188, 0.8879,  ..., 0.4169, 0.9626, 0.2554]])}

In [144]:
mvGraph.nodes()

tensor([     0,      1,      2,  ..., 209169, 209170, 209171])